In [10]:
from diseal import seal
import numpy as np

In [2]:
parms = seal.EncryptionParameters(seal.scheme_type.ckks)

poly_mod = 8192일 때, coeff_Modulus가 총합 얼마까지 가능? 

In [22]:
poly_modulus_degree = 8192*2
scale = 40

coeff_scales = [60] + [scale]*8 +[60]
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(seal.CoeffModulus.Create(poly_modulus_degree, coeff_scales))

In [6]:
context = seal.SEALContext(parms)
keygenerator = seal.KeyGenerator(context)
sk = keygenerator.secret_key()

In [7]:
pk = keygenerator.create_public_key()
gk = keygenerator.create_galois_keys()
rk = keygenerator.create_relin_keys()

In [9]:
encryptor = seal.Encryptor(context, pk)
evaluator = seal.Evaluator(context)
decryptor = seal.Decryptor(context, sk)
encoder   = seal.CKKSEncoder(context)

In [14]:
nslots = encoder.slot_count()

In [46]:
arr = np.random.rand(nslots)
print(arr)
ptxt = encoder.encode(arr, 2**scale)

[0.04884344 0.09937436 0.08188748 ... 0.53452455 0.72398056 0.14601238]


In [36]:
def decrypt(ctxt):
    decp = decryptor.decrypt(ctxt)
    return encoder.decode(decp)

def _sum_reduce_seal(evaluator, galois_keys, nslots, ctx: seal.Ciphertext, n=None):
    """return the sum of a Ciphertext (repeated nslot times), SEAL version
    """        
    if n is None:
        n = int(np.log2(nslots))

    temp = seal.Ciphertext()
    output = seal.Ciphertext()

    for i in range(n):
        if i == 0:
            temp = evaluator.rotate_vector(ctx, 2**i, galois_keys)
            output = evaluator.add(ctx, temp)
        else:
            temp = evaluator.rotate_vector(output, 2**i, galois_keys)
            evaluator.add_inplace(output, temp)
    return output

In [50]:
res = _sum_reduce_seal(evaluator, gk, nslots, ctxt)

In [53]:
dec = decrypt(res)

In [51]:
np.sum(arr)

4130.119460423652

In [54]:
print(dec[:30])

[4130.11945216 4130.11946023 4130.11945937 4130.11946065 4130.11946061
 4130.11945999 4130.1194585  4130.11946061 4130.11945972 4130.1194601
 4130.1194591  4130.11945946 4130.11946115 4130.1194604  4130.11945862
 4130.11946102 4130.11945759 4130.11945931 4130.11945867 4130.11946029
 4130.11946071 4130.11946011 4130.11945869 4130.11946001 4130.11946018
 4130.11945944 4130.11945946 4130.1194608  4130.11946101 4130.11946059]
